In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, array, explode, sum, count, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from datetime import datetime

SYMBOLS_LIST = ['BTC.X', 'BSV.X', 'BCH.X', 'LTC.X', 'ETH.X', 'DOGE.X']

# NOTE: The environment needs to have scala installed for this to work
spark = SparkSession \
.builder \
.appName("myApp") \
.config("spark.mongodb.input.uri", "mongodb://127.0.0.1/cryptoracle") \
.config("spark.mongodb.input.collection", "messages") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.0') \
.getOrCreate()

messages_df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

Py4JJavaError: An error occurred while calling o53.load.
: com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=127.0.0.1:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:179)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:41)
	at com.mongodb.client.internal.MongoClientDelegate.getServerAddressList(MongoClientDelegate.java:116)
	at com.mongodb.Mongo.getServerAddressList(Mongo.java:401)
	at com.mongodb.spark.connection.MongoClientCache.$anonfun$logClient$1(MongoClientCache.scala:161)
	at com.mongodb.spark.LoggingTrait.logInfo(LoggingTrait.scala:48)
	at com.mongodb.spark.LoggingTrait.logInfo$(LoggingTrait.scala:47)
	at c

In [ ]:
messages_df.count()

In [ ]:
messages_df.first()

### Create new dataframe with only required columns and filtered rows

In [ ]:
messages_df = messages_df.select(messages_df['_id']['oid'].alias('_id'),
                   messages_df['body'],
                   messages_df['created_at'],
                   messages_df['entities']['sentiment']['basic'].alias('sentiment'),
                   messages_df['symbols']['symbol'].alias('symbols'),
                   messages_df['likes']['total'].alias('likes'),
                   messages_df['reshares']['reshared_count'].alias('reshares'))

def sum_interactions(likes, reshares):
    _sum = 1
    if likes:
        _sum += likes
    if reshares:
        _sum += reshares
    return _sum

def convert_sentiment(sentiment):
    if sentiment == "Bullish":
        return 2
    if sentiment == "Bearish":
        return 1
    return 0

def convert_date(dt):
    return datetime.strptime(dt, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d-%H')

interactionUdf = udf(sum_interactions, IntegerType())
sentimentUdf = udf(convert_sentiment, IntegerType())
dateUdf = udf(convert_date, StringType())

messages_df = messages_df \
                .withColumn('interaction_count', interactionUdf('likes', 'reshares')) \
                .withColumn('sentiment', sentimentUdf('sentiment')) \
                .withColumn('created_at', dateUdf('created_at'))

In [ ]:
messages_df.first()

#### Distribution of sentiments

In [ ]:
messages_df.groupBy('sentiment').count().collect()

### Windowing


#### Unwrap array of symbols to new rows

In [ ]:
# Since tweets can be attributed to more than one symbol, we unwrap the list into more rows
# this is okay as our final calculation will be grouped by symbol among other things

def weight_sentiment(sentiment, count):
    if sentiment == 0:
        return 0
    return sentiment * count

def handle_neutral_sentiment(sentiment, count):
    if sentiment == 0:
        return 0
    return count

weightedSentimentUdf = udf(weight_sentiment, IntegerType())
neutralSentimentUdf = udf(handle_neutral_sentiment, IntegerType())


messages_df = messages_df \
                .withColumn('weighted_sentiment', weightedSentimentUdf('sentiment', 'interaction_count')) \
                .withColumn('symbol', explode(messages_df['symbols']))

# remove duplicates after exploding
messages_df = messages_df.dropDuplicates()

# do not consider interaction count for neutral sentiment
messages_df = messages_df.withColumn('interaction_count', neutralSentimentUdf('sentiment', 'interaction_count'))
# filter to only those symbols that we care about
messages_df = messages_df.where(messages_df['symbol'].isin(SYMBOLS_LIST))

#### Include counts of sentiment for later aggr in groupby

In [ ]:
def positive_sentiment_count(sentiment):
    return (sentiment == 2) ? 1 : 0
    
def negative_sentiment_count(sentiment):
    return (sentiment == 1) ? 1 : 0
    
def null_sentiment_count(sentiment):
    return (sentiment == 0) ? 1 : 0


positiveSentimentCountUdf = udf(positive_sentiment_count, IntegerType())
negativeSentimentCountUdf = udf(negative_sentiment_count, IntegerType())
nullSentimentCountUdf = udf(null_sentiment_count, IntegerType())

messages_df = messages_df \
                .withColumn('positive_count', positiveSentimentCountUdf('sentiment')) \
                .withColumn('negative_count', negativeSentimentCountUdf('sentiment')) \
                .withColumn('null_count', nullSentimentCountUdf('sentiment')) 

In [ ]:
messages_df.groupby('symbol').count().collect()

In [ ]:
grouped_df = messages_df.groupby(['created_at', 'symbol']).agg(
    sum('interaction_count').alias('sum_interaction_count'),
    sum('weighted_sentiment').alias('sum_weighted_sentiment'),
    sum('positive_count').alias('sum_positive_count'),
    sum('negative_count').alias('sum_negative_count'),
    sum('null_count').alias('sum_null_count'),
    count('_id').alias('volume_tweets'))

grouped_df = grouped_df.withColumn('overall_sentiment', grouped_df['sum_weighted_sentiment'] / grouped_df['sum_interaction_count'])

In [ ]:
gdf = grouped_df.toPandas()

In [ ]:
gdf[gdf['created_at'] == '2020-03-24-23']

In [ ]:
grouped_df.groupby('symbol').count().collect()

## Historical Price Data

In [ ]:
import pandas as pd 
from datetime import datetime
read_func = lambda x: spark.read.format('csv').load(x, header=True, inferSchema=True)

def format_date(date, sym):
    if sym in ['BCH.X', 'DOGE.X']:
        return datetime.strptime(date, '%Y-%m-%d %I-%p').strftime("%Y-%m-%d-%H")
    return datetime.strptime(date, '%m/%d/%y %H:%M').strftime("%Y-%m-%d-%H")
    

formatDateUdf = udf(format_date, StringType())

BTC_prices = read_func("data/gemini_BTCUSD_1hr.csv") 
ETH_prices = read_func("data/gemini_ETHUSD_1hr.csv") 
LTC_prices = read_func("data/gemini_LTCUSD_1hr.csv") 
BCH_prices = read_func("data/Bitbay_BCHUSD_1h.csv") 
DOGE_prices = read_func("data/Yobit_DOGERUR_1h.csv") 

price_df_lists = [
    ('BTC.X', BTC_prices),
    ('ETH.X', ETH_prices),
    ('LTC.X', LTC_prices),
    ('BCH.X', BCH_prices),
    ('DOGE.X', DOGE_prices)
]

#### 1. Datetime and Symbols

In [ ]:
new_dfs = []
for sym, price_df in price_df_lists:
    new_dfs.append(price_df \
                .withColumnRenamed('Symbol', 'symbol') \
                .withColumn('symbol', lit(sym)) \
                .withColumn('Date', formatDateUdf('Date', 'symbol')) \
                .drop('Unix Timestamp'))
price_df_lists = new_dfs

#### 2. Doge (RUR to USD)

In [ ]:
RUR_USD = 0.013

def convert_doge(price):
    return price * RUR_USD

convertPriceUdf = udf(convert_doge, DoubleType())

for col in ['Open', 'High', 'Low', 'Close']:
    price_df_lists[-1] = price_df_lists[-1].withColumn(col, convertPriceUdf(col))
    
#DOGE
price_df_lists[-1] = price_df_lists[-1].drop('Volume ERUR').withColumnRenamed('Volume DOG', 'Volume')
#BCH
price_df_lists[-2] = price_df_lists[-2].drop('Volume USD').withColumnRenamed('Volume BCH', 'Volume')

In [ ]:
for df in price_df_lists:
    print(df.first())

In [ ]:
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

unionAll(*price_df_lists).count()

In [ ]:
spark.stop()
